In [ ]:
# load libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.model_selection import RepeatedStratifiedKFold,StratifiedKFold,KFold
pd.set_option('display.max_columns', 85)
pd.set_option('display.max_rows',85)

In [4]:
#data
train=pd.read_csv("wids2_train.csv")
test=pd.read_csv("wids2_test.csv")
ss=pd.read_csv("solution_template.csv")

In [5]:
train.head()

,patient_id,patient_race,payer_type,patient_state,patient_zip3,Region,Division,patient_age,patient_gender,bmi,breast_cancer_diagnosis_code,breast_cancer_diagnosis_desc,metastatic_cancer_diagnosis_code,metastatic_first_novel_treatment,metastatic_first_novel_treatment_type,population,density,age_median,age_under_10,age_10_to_19,age_20s,age_30s,age_40s,age_50s,age_60s,age_70s,age_over_80,male,female,married,divorced,never_married,widowed,family_size,family_dual_income,income_household_median,income_household_under_5,income_household_5_to_10,income_household_10_to_15,income_household_15_to_20,income_household_20_to_25,income_household_25_to_35,...,Average of Aug-15,Average of Sep-15,Average of Oct-15,Average of Nov-15,Average of Dec-15,Average of Jan-16,Average of Feb-16,Average of Mar-16,Average of Apr-16,Average of May-16,Average of Jun-16,Average of Jul-16,Average of Aug-16,Average of Sep-16,Average of Oct-16,Average of Nov-16,Average of Dec-16,Average of Jan-17,Average of Feb-17,Average of Mar-17,Average of Apr-17,Average of May-17,Average of Jun-17,Average of Jul-17,Average of Aug-17,Average of Sep-17,Average of Oct-17,Average of Nov-17,Average of Dec-17,Average of Jan-18,Average of Feb-18,Average of Mar-18,Average of Apr-18,Average of May-18,Average of Jun-18,Average of Jul-18,Average of Aug-18,Average of Sep-18,Average of Oct-18,Average of Nov-18,Average of Dec-18,metastatic_diagnosis_period
0,268700,NaN,COMMERCIAL,AR,724,South,West South Central,39,F,NaN,C50912,Malignant neoplasm of unspecified site of left...,C773,NaN,NaN,3924.87,82.63,42.58,11.61,13.03,10.87,11.80,12.29,13.22,13.47,10.07,3.64,51.43,48.57,51.05,16.72,23.57,8.66,3.01,43.99,44483.35,2.21,3.97,8.52,7.08,7.67,13.82,...,76.28,73.32,60.32,51.00,48.37,35.76,42.89,53.21,61.07,66.29,82.78,81.52,79.52,75.61,65.76,53.85,39.56,41.59,49.01,52.28,63.40,67.11,75.78,80.58,76.16,72.91,62.68,49.73,39.80,31.71,41.69,50.22,52.55,74.77,79.96,81.69,78.30,74.56,59.98,42.98,41.18,191
1,484983,White,NaN,IL,629,Midwest,East North Central,55,F,35.36,C50412,Malig neoplasm of upper-outer quadrant of left...,C773,NaN,NaN,2745.39,51.79,43.54,11.22,12.19,11.45,11.01,11.35,14.39,14.15,9.17,5.05,49.32,50.68,49.48,15.42,26.93,8.17,3.17,41.41,51796.79,3.67,3.86,6.58,5.58,5.38,11.02,...,73.88,70.78,58.70,50.50,46.71,32.36,38.76,51.68,58.41,64.10,78.15,79.70,78.33,72.80,63.84,50.35,34.90,39.05,45.80,48.45,60.77,65.89,74.46,79.18,73.13,70.13,59.68,46.52,35.15,28.43,40.01,44.18,49.30,72.87,77.40,77.43,75.83,72.64,58.36,39.68,39.71,33
2,277055,NaN,COMMERCIAL,CA,925,West,Pacific,59,F,NaN,1749,"Malignant neoplasm of breast (female), unspeci...",C773,NaN,NaN,38343.18,700.34,36.28,13.27,15.66,13.49,13.45,12.40,11.58,10.47,6.38,3.28,49.99,50.01,48.81,11.90,34.35,4.95,3.80,52.89,78696.87,2.59,1.81,3.16,3.71,3.23,7.40,...,85.08,82.01,75.16,59.59,53.24,54.54,63.52,63.72,67.45,69.58,82.02,84.83,84.06,77.85,72.50,64.24,55.52,53.54,58.18,65.45,68.51,71.13,80.71,85.64,84.85,78.14,74.03,65.70,59.28,60.34,57.43,60.97,68.50,70.31,78.61,87.24,85.52,80.75,70.81,62.67,55.58,157
3,320055,Hispanic,MEDICAID,CA,900,West,Pacific,59,F,NaN,C50911,Malignant neoplasm of unsp site of right femal...,C773,NaN,NaN,36054.12,5294.33,36.65,9.76,11.27,17.23,17.44,13.09,12.30,9.41,5.67,3.82,50.51,49.49,33.48,11.30,50.46,4.77,3.44,55.53,69266.69,6.32,2.95,6.81,4.18,4.13,7.84,...,77.47,76.77,74.50,63.48,57.98,57.69,66.53,61.89,64.86,62.63,70.45,73.09,71.94,72.26,69.56,66.85,58.29,56.08,57.88,63.84,65.03,63.90,68.95,73.58,73.62,72.88,73.84,68.87,66.63,63.75,59.89,59.35,63.34,63.10,67.45,75.86,75.24,71.10,68.95,65.46,59.46,146
4,190386,NaN,COMMERCIAL,CA,934,West,Pacific,71,F,NaN,1748,Malignant neoplasm of other specified sites of...,C7951,NaN,NaN,13700.37,400.48,41.78,10.03,16.43,12.97,11.29,10.09,11.56,13.28,8.78,5.53,51.99,48.01,48.21,11.16,35.60,5.01,3.09,51.88,82314.64,2.74,1.21,2.56,2.59,4.37,6.65,...,69.57,71.37,68.40,55.45,49.49,54.15,60.98,58.44,62.06,61.68,70.15,69.49,66.71,68.10,64.41,59.72,52.52,51.80,54.20,59.36,60.44,62.4

In [6]:
y=train['metastatic_diagnosis_period'] #target
data=pd.concat((train,test)).reset_index(drop=True).copy()
data=data.drop(columns=['patient_id','metastatic_diagnosis_period'],axis=1)

In [7]:
#engineer some features
data["clust"]=(data.metastatic_cancer_diagnosis_code.str.len()==4).astype("int")
data["is_female"] = data.breast_cancer_diagnosis_desc.str.contains("female").astype("int")

In [8]:
# Get the columns with object datatype
object_cols = data.select_dtypes(include=['object']).columns

# Convert the object columns to category dtype
for col in object_cols:
    #data[col] = data[col].astype('category').fillna("Missing")
    data[col] = pd.Categorical(data[col].fillna("Missing"))
    #data[col] = data[col].fillna("Missing")

In [9]:
#separate train and test data
train=data[:len(train)]
test=data[len(train):]

In [10]:
features=['breast_cancer_diagnosis_code','patient_age','metastatic_cancer_diagnosis_code','patient_race',
   
   'payer_type','breast_cancer_diagnosis_desc','bmi','Division','patient_state']

In [ ]:
scores = []
final_prediction=[]
kf = KFold(n_splits=15, random_state=42, shuffle=True)
#kf = StratifiedKFold(n_splits=20, random_state=42, shuffle=True)
for train_index, test_index in kf.split(train,y):
    X_train, X_test = train.loc[train_index], train.loc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clf=CatBoostRegressor(verbose=False,eval_metric='RMSE')

    clf.fit(X_train[features], y_train,cat_features=['breast_cancer_diagnosis_code','metastatic_cancer_diagnosis_code',
                                              'patient_race',
   
   'payer_type','breast_cancer_diagnosis_desc','Division','patient_state'],
            
            eval_set=(X_test[features], y_test),use_best_model=True,early_stopping_rounds=500)
    
    preds = clf.predict(X_test[features])
                             
    test_preds=clf.predict(test[features])
                             
    final_prediction.append(test_preds)
    score = mean_squared_error(y_test, preds,squared=False)
    print("RMSE:", score)
    scores.append(score)

In [ ]:
np.mean(scores)

In [12]:
#stack final prediction and get it's mean
prediction=np.mean(np.column_stack(final_prediction),axis=1)

In [13]:
prediction

array([223.21205396,  59.6603403 , 199.83096887, ..., 237.12750395,
        65.43472407,  52.13038093])

In [17]:
#sample submit
ss.head()

,patient_id,metastatic_diagnosis_period
0,730681,223.212054
1,334212,59.660340
2,571362,199.830969
3,907331,214.896197
4,208382,46.065446


In [15]:
ss['metastatic_diagnosis_period']= prediction

In [16]:
#generate submission file
ss.to_csv('catboost_baseline.csv', index=False)